In [13]:
import math
import torch

In [14]:
num_steps = 8

In [15]:
for k in range(0, num_steps-4):
    print(k)

print(num_steps-4)
print(num_steps-3)
print(num_steps-2)
print(num_steps-1)

0
1
2
3
4
5
6
7


In [341]:
def unfolded(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps-6):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]

    # num_steps-6
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-5
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-4
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-3
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-2
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

    # num_steps-1
    T = 2*(Xa.size(0) // 2)
    Xa = Xa[:T].view(T//2, 2)
    Xa[:, 1].add_(Xa[:, 0])
    Xa = Xa[:, 1]

def unfolded_3(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps-3):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    Xa[1].add_(Xa[0])
    Xa[3].add_(Xa[2])
    Xa[5].add_(Xa[4])
    Xa[7].add_(Xa[6])

    Xa[3].add_(Xa[1])
    Xa[7].add_(Xa[5])

    Xa[7].add_(Xa[3])

def unfolded_4(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps-3):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    Xa[1].add_(Xa[0])
    Xa[3].add_(Xa[1]+Xa[2])
    Xa[5].add_(Xa[4])
    Xa[7].add_(Xa[3]+X[5]+X[6])

### BEST ###
def unfolded_2(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for _ in range(num_steps-2):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]
    
    # the last 2 steps are unfolded/done manually (up to 7 nodes)
    T = 2 * (Xa.size(0) // 2)
    if T == 6:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
        Xa[5].add_(Xa[4])
    elif T == 4:
        Xa[1].add_(Xa[0])
        Xa[3].add_(Xa[2].add(Xa[1]))
    elif T == 3:
        Xa[1].add_(Xa[0])
    elif T == 2:
        Xa[1].add_(Xa[0])

def normal(X):
    Xa = X
    num_steps = int(math.log2(X.size(0)))

    for k in range(num_steps):
        T = 2*(Xa.size(0)//2)

        Xa = Xa[:T].view(T//2, 2)
        Xa[:, 1].add_(Xa[:, 0])
        Xa = Xa[:, 1]

In [342]:
L = 8
X = torch.arange(L).to("cuda")
X

tensor([0, 1, 2, 3, 4, 5, 6, 7], device='cuda:0')

In [343]:
X = torch.arange(L).to("cuda")
normal(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [344]:
X = torch.arange(L).to("cuda")
unfolded_2(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [204]:
X = torch.arange(L).to("cuda")
unfolded_3(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [205]:
X = torch.arange(L).to("cuda")
unfolded_4(X)
X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28], device='cuda:0')

In [348]:
L = 512
X = torch.arange(L).to("cuda")

In [349]:
%timeit normal(X)

198 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [350]:
%timeit unfolded_2(X)

181 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [219]:
%timeit unfolded_3(X)

226 µs ± 6.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [220]:
%timeit unfolded_4(X)

225 µs ± 4.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [168]:
X = torch.arange(4)
X

tensor([0, 1, 2, 3])

In [169]:
X[1].add_(X[0])
X[3].add_(X[2].add(X[1]))
X

tensor([0, 1, 2, 6])

In [177]:
X = torch.arange(8)
X

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [178]:
X[1].add_(X[0])
X[3].add_(X[2])
X[5].add_(X[4])
X[7].add_(X[6])

X[3].add_(X[1])
X[7].add_(X[5])

X[7].add_(X[3])

X

tensor([ 0,  1,  2,  6,  4,  9,  6, 28])

In [108]:
%timeit normal(X)

121 µs ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [99]:
%timeit unfolded(X)

249 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
